исходно - учебный проект Гадамян Лиды (Ghadamiyan Lida, class 407 AI)

In [ ]:
import datetime
import re
import json

In [ ]:
from IPython.display import Image, display_html

In [ ]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
#nltk.download('wordnet')
#nltk.download('punkt')
stemmer = SnowballStemmer('russian')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score

import seaborn as sns

from yellowbrick.text import FreqDistVisualizer, TSNEVisualizer

from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

## Data Preprocessing

In [ ]:
# data = pd.read_csv('../input/russian-language-toxic-comments/labeled.csv')

Извлечём данные для поста https://vk.com/wall-72378974_6807820

In [ ]:
opo = json.loads('''{
 "type": "post",
 "id": "http://tombi/communa/vk/-72378974/post/6807820/",
 "u": "https://vk.com/wall-72378974_6807820",
 "post_id": 6807820,
 "t": "2021-02-26T16:28:48.181482",
 "from_id": -72378974,
 "owner_id": -72378974,
 "date": 1614340860,
 "marked_as_ads": 0,
 "post_type": "post",
 "text": "Хорошая попытка, Nvidia, но не получилось.",
 "attachments": [
  {
   "type": "photo",
   "photo": {
    "album_id": -7,
    "date": 1614328813,
    "id": 458113183,
    "owner_id": -72378974,
    "has_tags": false,
    "access_key": "a9afa0775ae9941749",
    "post_id": 6807820,
    "sizes": [
     {
      "height": 130,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=96x130&quality=96&sign=f8879041a82ea6afb89179e543624a00&c_uniq_tag=Xe1Kzsuj2s-tAMkEBIhUwLdmNG1nwX70aCBy_58ZZko&type=album",
      "type": "m",
      "width": 96
     },
     {
      "height": 175,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=130x175&quality=96&sign=fba85c7bed05833320060e6150383206&c_uniq_tag=b5pxoZBH7GgSdDmiqCox_ChiyTukhaMoax0hghDZnLo&type=album",
      "type": "o",
      "width": 130
     },
     {
      "height": 270,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=200x270&quality=96&sign=3b5c369887fdb2bce6024b23222b5527&c_uniq_tag=ysRTAEmQCeYBDbEEnyTtFMLhxeDhZtOyakH3PlT9g4w&type=album",
      "type": "p",
      "width": 200
     },
     {
      "height": 432,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=320x432&quality=96&sign=af9ea078133fe437840841d7f536ffcd&c_uniq_tag=pBG1uY-yJvEW8renxhczDgGGngVnDfQvTKGQwDYcu9o&type=album",
      "type": "q",
      "width": 320
     },
     {
      "height": 689,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=510x688&quality=96&sign=72d937df55bdc202ba198385ae3fee82&c_uniq_tag=MbDzELXwk-g0Y7SNXIeMzPlB6tJsWfh374GaLa4Ztvc&type=album",
      "type": "r",
      "width": 510
     },
     {
      "height": 75,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=56x75&quality=96&sign=c637e2f014297c9a4525609161b30949&c_uniq_tag=dPLrsgsBwA9jObUIc1mvWwzoyV1-psN-oxOoTlqd0Dw&type=album",
      "type": "s",
      "width": 55
     },
     {
      "height": 1417,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=1050x1417&quality=96&sign=fefc6ed03372a9148c534cdcdecda4e6&c_uniq_tag=45bEI8sC1fgXb3AS2JWyse71WbG-_691G6k6ZZRFdYY&type=album",
      "type": "w",
      "width": 1050
     },
     {
      "height": 604,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=448x604&quality=96&sign=a986277ba4b9e1ee4713bbca48e5d5a3&c_uniq_tag=jmKFqxJuh2OJ67tzocy3MiO626518y53s6xMrKxtz8U&type=album",
      "type": "x",
      "width": 448
     },
     {
      "height": 807,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=598x807&quality=96&sign=7d57a46441e703349b96e047ed35b982&c_uniq_tag=D5MrxcTTgeo3D5uM9f6L9wsQPaDmYEgVFiGtURd-JpM&type=album",
      "type": "y",
      "width": 598
     },
     {
      "height": 1080,
      "url": "https://sun1-93.userapi.com/impg/uFBj3jt2uxyEV1R0jaM9xR7nTXVno677S-lzOg/fXUO1y3zEUY.jpg?size=800x1080&quality=96&sign=fe913d776b02dbb76cb1548d55c0a99d&c_uniq_tag=PraQeEe8HgGC2W6azsWKk363OOqbnyap-RbG5ElwRDM&type=album",
      "type": "z",
      "width": 800
     }
    ],
    "text": "",
    "user_id": 100
   }
  }
 ],
 "post_source": {
  "type": "vk"
 },
 "comments": {
  "count": 197,
  "can_post": 1,
  "groups_can_post": true
 },
 "likes": {
  "count": 438,
  "user_likes": 0,
  "can_like": 1,
  "can_publish": 1,
  "t": "2021-02-26T16:28:48.181482"
 },
 "reposts": {
  "count": 379,
  "user_reposted": 0
 },
 "views": {
  "count": 36712,
  "t": "2021-02-26T16:28:48.181482"
 },
 "is_favorite": false,
 "donut": {
  "is_donut": false
 },
 "short_text_rate": 0.8
}''')

pd.Timestamp(opo['date'], unit='s'), datetime.datetime.fromtimestamp(opo['date']), opo['u']

In [ ]:
oco = json.loads(r'''{
 "type": "sarcina",
 "id": "http://tombi/communa/vk/-72378974/post/6807820/comments",
 "count": 198,
 "items": [
  {
   "id": 6807830,
   "from_id": 363015644,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341022,
   "text": "Этого следовало ожидать.",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807831,
   "from_id": 147559417,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341031,
   "text": "тварьнеры.",
   "likes": {
    "count": 4,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614341031,
      "id": 458113363,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "ca47baf2378db08f78",
      "sizes": [
       {
        "height": 126,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=130x126&quality=96&sign=f79304ebe42518a1db00c12fd423924b&c_uniq_tag=TrsU0Cz-UOiLwJlXbzimFUSe4RTXN8yXTYQF7QE-mHg&type=album",
        "type": "m",
        "width": 130
       },
       {
        "height": 126,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=130x126&quality=96&sign=f79304ebe42518a1db00c12fd423924b&c_uniq_tag=TrsU0Cz-UOiLwJlXbzimFUSe4RTXN8yXTYQF7QE-mHg&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 193,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=200x193&quality=96&sign=ea40a46cdf39d554e5e67379216f1383&c_uniq_tag=Ag00pKnCG2oTXwa-i8rv1yF6cUPjlkK7JEeVIPgba7Q&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 309,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=320x309&quality=96&sign=56d79804a670c34ab5c171693ff8a69a&c_uniq_tag=BU8TGJPKq-UNYCc-TY-ZVKSNapSfDYLkshFoQ0QqOV8&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 493,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=510x493&quality=96&sign=f8f01f81267bf5e222bd29ef874e3be7&c_uniq_tag=uc58S_N3dJPlX-GmFZfqr5vEu6wQYSZDH-oBFieng3o&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 73,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=75x73&quality=96&sign=2bc65b4b7c6d44d8bd8de7f62676868d&c_uniq_tag=0EaJI0jpARPgd6FwlAW02gNfk9Z5KODa--9_XIX5LXg&type=album",
        "type": "s",
        "width": 75
       },
       {
        "height": 584,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=604x584&quality=96&sign=cb9a05afe8252ddaba476820e8696c6f&c_uniq_tag=PJTgHZiT582Jg9PBuWo2nn02Kb9r_Yw36OLLMhKm1bQ&type=album",
        "type": "x",
        "width": 604
       },
       {
        "height": 696,
        "url": "https://sun1-30.userapi.com/impg/JOuD-kiD7fZclPC9ejTdlZlHXmB6xFYgt_gxCA/eXNEud5bVe0.jpg?size=720x696&quality=96&sign=00863553be94d2d1ff03301dd8a5d141&c_uniq_tag=iddgeSsrIL49N-bAnoTTbJmB5PtZE_qwoe0KyLoqLBo&type=album",
        "type": "y",
        "width": 720
       }
      ],
      "text": "",
      "user_id": 147559417
     }
    }
   ],
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807832,
   "from_id": 86378088,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341036,
   "text": "и не получится, только майка может вставить палки в колёсса майнинга",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 2,
    "items": [
     {
      "id": 6807857,
      "from_id": 160644108,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807832
      ],
      "date": 1614341315,
      "text": "[id86378088|Рома], крупные майнеры и без винды обходятся, только ламеров если обломать, которые карты эти не скупают и майнят обычно на своих же",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 86378088,
      "reply_to_comment": 6807832
     },
     {
      "id": 6807904,
      "from_id": 145791250,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807832
      ],
      "date": 1614341814,
      "text": "[id86378088|Рома], обычно фермы на линуксе работают",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 86378088,
      "reply_to_comment": 6807832
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807834,
   "from_id": 363015644,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341101,
   "text": "Короче гейминг умрёт.",
   "likes": {
    "count": 7,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 5,
    "items": [
     {
      "id": 6807927,
      "from_id": 197025871,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807834
      ],
      "date": 1614342102,
      "text": "[id363015644|Алекс], с таким раскладом скоро Денди и Сега завоюют мир 🔥",
      "likes": {
       "count": 9,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 363015644,
      "reply_to_comment": 6807834
     },
     {
      "id": 6808024,
      "from_id": 479628381,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807834
      ],
      "date": 1614342958,
      "text": "[id197025871|Евгений], на них будут какую то особую крипту майнить",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 197025871,
      "reply_to_comment": 6807927
     },
     {
      "id": 6808052,
      "from_id": 2547354,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807834
      ],
      "date": 1614343218,
      "text": "[id479628381|Данила], харвестерами",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 479628381,
      "reply_to_comment": 6808024
     },
     {
      "id": 6808099,
      "from_id": 197025871,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807834
      ],
      "date": 1614343685,
      "text": "[id479628381|Данила], мля..... Реально... Хахахха.... Дилдокоины 😁🔥",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 479628381,
      "reply_to_comment": 6808024
     },
     {
      "id": 6808270,
      "from_id": 94345860,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807834
      ],
      "date": 1614345868,
      "text": "[id363015644|Алекс], считай многие на слабых, или без видюх, и выпускать не выгодно уже на пк. Тока если порты выпускать, после майнинга.",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 363015644,
      "reply_to_comment": 6807834
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807835,
   "from_id": 175652535,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341114,
   "text": "А стоимость электричества учитывалась в расчётах окупаемости?",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 12,
    "items": [
     {
      "id": 6807845,
      "from_id": 251924757,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341212,
      "text": "[id175652535|Максим], шо вы доебались до электричества. Там пару дней майнинга отбивает месяц электричества",
      "likes": {
       "count": 29,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 175652535,
      "reply_to_comment": 6807835
     },
     {
      "id": 6807847,
      "from_id": 444529,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341226,
      "text": "[id175652535|Максим], там копейки уходят",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 175652535,
      "reply_to_comment": 6807835
     },
     {
      "id": 6807865,
      "from_id": 209088184,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341430,
      "text": "[id444529|Александр], в России копейки да",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 444529,
      "reply_to_comment": 6807847
     },
     {
      "id": 6807870,
      "from_id": 621798090,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341468,
      "text": "[id444529|Александр], смотря в какой стране,у нас действительно копейки",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 444529,
      "reply_to_comment": 6807847
     },
     {
      "id": 6807886,
      "from_id": 444529,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341679,
      "text": "[id621798090|Ghost], ну в Китае так же как и у нас)",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621798090,
      "reply_to_comment": 6807870
     },
     {
      "id": 6807909,
      "from_id": 74429847,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614341874,
      "text": "[id175652535|Максим], да",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 175652535,
      "reply_to_comment": 6807835
     },
     {
      "id": 6807974,
      "from_id": 209088184,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614342510,
      "text": "[id621798090|Ghost], в Италии ты будешь майнить только за электричества",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621798090,
      "reply_to_comment": 6807870
     },
     {
      "id": 6808073,
      "from_id": 621798090,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614343464,
      "text": "[id444529|Александр], не знал, благодарю за информацию)",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 444529,
      "reply_to_comment": 6807886
     },
     {
      "id": 6808075,
      "from_id": 621798090,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614343494,
      "text": "[id209088184|Orif], так дорого?",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 209088184,
      "reply_to_comment": 6807974
     },
     {
      "id": 6808086,
      "from_id": 209088184,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807835
      ],
      "date": 1614343615,
      "text": "[id621798090|Ghost], в Европе через 10 лет начнутся такие большие проблемы с электричеством \nЧто не могу представить",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621798090,
      "reply_to_comment": 6808075
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807836,
   "from_id": 621847649,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341124,
   "text": "Закупили 20 штук 3060 майнится без проблем .",
   "likes": {
    "count": 16,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 88,
    "items": [
     {
      "id": 6807873,
      "from_id": 341923672,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341473,
      "text": "[id621847649|Олег], чтоб у тя ферма сгорела",
      "likes": {
       "count": 118,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621847649,
      "reply_to_comment": 6807836
     },
     {
      "id": 6807876,
      "from_id": 547734948,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341525,
      "text": "[id341923672|Андрей], че такого хотим майним хотим нет",
      "likes": {
       "count": 8,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 341923672,
      "reply_to_comment": 6807873
     },
     {
      "id": 6807880,
      "from_id": 621847649,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341597,
      "text": "[id341923672|Андрей], это врятли , противопожарная система все дела ... окупаемость быстрее чем пишут ... на самом деле .) месяц максимум.)",
      "likes": {
       "count": 6,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 341923672,
      "reply_to_comment": 6807873
     },
     {
      "id": 6807902,
      "from_id": 341923672,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341794,
      "text": "[id547734948|Абай], а я видюху хочу по адекватной цене",
      "likes": {
       "count": 3,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 547734948,
      "reply_to_comment": 6807876
     },
     {
      "id": 6807905,
      "from_id": 621847649,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341830,
      "text": "[id341923672|Андрей], потерпи до конца года .)",
      "likes": {
       "count": 2,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 341923672,
      "reply_to_comment": 6807902
     },
     {
      "id": 6807911,
      "from_id": 303033987,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614341887,
      "text": "[id621847649|Олег], почём взяли",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621847649,
      "reply_to_comment": 6807836
     },
     {
      "id": 6807920,
      "from_id": 243502846,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614342003,
      "text": "[id621847649|Олег], уебище",
      "likes": {
       "count": 14,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621847649,
      "reply_to_comment": 6807836
     },
     {
      "id": 6807922,
      "from_id": 172172707,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614342011,
      "text": "[id621847649|Олег], и какая скорость у одной ? И энергопотребление ?",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621847649,
      "reply_to_comment": 6807836
     },
     {
      "id": 6807923,
      "from_id": 621847649,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614342047,
      "text": "[id303033987|Иван], 30 с копейками",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 303033987,
      "reply_to_comment": 6807911
     },
     {
      "id": 6807934,
      "from_id": 303033987,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807836
      ],
      "date": 1614342183,
      "text": "[id621847649|Олег], ого а где взял так",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 621847649,
      "reply_to_comment": 6807923
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807839,
   "parents_stack": [],
   "date": 1614341155,
   "thread": {
    "count": 14,
    "items": [
     {
      "id": 6807849,
      "from_id": 363015644,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341244,
      "text": "[id217142688|Артём], Цена карты в 27к?",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807851,
      "from_id": 98604084,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341265,
      "text": "[id217142688|Артём], они думают, что она стоит 27к рублей :)",
      "likes": {
       "count": 5,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807855,
      "from_id": 27663824,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341304,
      "text": "[id217142688|Артём], видимо за 27000 покупали видяху)))",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807858,
      "from_id": 12566889,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341319,
      "text": "[id217142688|Артём], тут имелось в виду в доларах и цена карты также в доларах",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807868,
      "from_id": 27663824,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341449,
      "text": "[id12566889|Василий], какая разница в долларах или в чем считать\nОна стоит 60к это уже полгода, а то и больше",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 12566889,
      "reply_to_comment": 6807858
     },
     {
      "id": 6807878,
      "from_id": 12566889,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341584,
      "text": "[id27663824|Александр], а в том что там она стоит в раене 330доларов и 4бакза на 90дней   будит 360",
      "likes": {
       "count": 3,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 27663824,
      "reply_to_comment": 6807868
     },
     {
      "id": 6807881,
      "from_id": 256840474,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341604,
      "text": "[id27663824|Александр], карта стоит 330$, 4*3 месяца = 360 баксов",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 27663824,
      "reply_to_comment": 6807868
     },
     {
      "id": 6807890,
      "from_id": 145791250,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341712,
      "text": "[id217142688|Артём], так лол майнеры оптом прямо на заводах видюхи покупают",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807896,
      "from_id": 744775,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614341749,
      "text": "[id217142688|Артём], рекомендованная цена со старта 330 бачей, бОльшая часть майнеров предзаказывала карты по этой стоимости. умножь 4 на 90 и получишь как раз ту самую окупаемость. Нет смысла конвертировать это в рубли, если все равно цена привязана к баксу и закупки были не через розничные магазины.",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     },
     {
      "id": 6807929,
      "from_id": 413430073,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807839
      ],
      "date": 1614342118,
      "text": "[id217142688|Артём], цена на 3060 в Европе стартует от 330$",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 217142688,
      "reply_to_comment": 6807839
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   },
   "deleted": true
  },
  {
   "id": 6807844,
   "from_id": 311405548,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341189,
   "text": "да забудьте вы про видеокарты узбагойтись их не буит пару лет",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807848,
   "from_id": 311405548,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341228,
   "text": "я се скупил и то что будут производить тоже купил",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807863,
   "from_id": 137179703,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341408,
   "text": "Все уже давно это знали...",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807864,
   "from_id": -190334605,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341409,
   "text": "",
   "likes": {
    "count": 16,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614341409,
      "id": 457239932,
      "owner_id": -190334605,
      "has_tags": false,
      "access_key": "37572e2f931a6fc42c",
      "sizes": [
       {
        "height": 105,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=130x105&quality=96&sign=27bd36918635905f6d3c7e40b73a28a5&c_uniq_tag=F7ZXDApoivT0GzAv5JNEp5Art_d_fsi-GuSBqjdnJTE&type=album",
        "type": "m",
        "width": 130
       },
       {
        "height": 105,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=130x105&quality=96&sign=27bd36918635905f6d3c7e40b73a28a5&c_uniq_tag=F7ZXDApoivT0GzAv5JNEp5Art_d_fsi-GuSBqjdnJTE&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 161,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=200x161&quality=96&sign=f38f94ff5ad1e7ff7ef9acb24b62e800&c_uniq_tag=fm7KEIn7uCIemVmq9LM-LKWNraqAyfUuaSgvojIxPhs&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 258,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=320x258&quality=96&sign=46d750c779c26291043671e94528e03e&c_uniq_tag=T0Hq2zxcv87ONydWEV9FU6aUv7Io0NulVfNWZJ4ZDhw&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 411,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=510x411&quality=96&sign=be818c22e2155950185180dff4d2a722&c_uniq_tag=Mmosu3-8f99lWF7TUyFkOdHAmKMMkhWzdqrcZwRUMGM&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 61,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=75x60&quality=96&sign=160cd638e20109d4201adb6f90269483&c_uniq_tag=wV3vmWb0gWRrOwMSYgFNLV8qJMxVxS18dajW1jwUzUE&type=album",
        "type": "s",
        "width": 75
       },
       {
        "height": 487,
        "url": "https://sun9-74.userapi.com/impf/6foLhhNyziZCNSGxtiYMKxhLjcKmqEFNdB1DEA/Z-kbIe17-sM.jpg?size=604x487&quality=96&sign=662cccb17b8d9691edf794d2e47ed2d3&c_uniq_tag=5Zgyqim6jW0vrJI6fKJDwfMjzqB80pw0vB2tk-DlDpo&type=album",
        "type": "x",
        "width": 604
       }
      ],
      "text": "",
      "user_id": 100
     }
    }
   ],
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6807991,
      "from_id": 143706170,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807864
      ],
      "date": 1614342629,
      "text": "[club190334605|Testix🅥 – Ванильный сервер Minecraft PE],  иди в Майнкрафт играй",
      "likes": {
       "count": 4,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": -190334605,
      "reply_to_comment": 6807864
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807867,
   "from_id": 341923672,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341447,
   "text": "",
   "likes": {
    "count": 3,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614341447,
      "id": 458113371,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "d326d159555f0125e7",
      "sizes": [
       {
        "height": 17,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=130x17&quality=96&sign=d0e874fecf1fd68843c3a46bd9631695&c_uniq_tag=4W1CNdDRw0pyO5YlszYcLl9NTrDy617-WllXGHLwBVY&type=album",
        "type": "m",
        "width": 130
       },
       {
        "height": 46,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=130x46&quality=96&crop=110,0,130,46&sign=fb3031ec89dfbf533a0c4d3a9dbfa9e1&c_uniq_tag=WCKWvs0VnXYbOHJG-5unxgLjlZ05UhVodNChOlKiG14&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 46,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=200x46&quality=96&crop=75,0,200,46&sign=8fcc6fbff7c5d71e79e6c30db01d7381&c_uniq_tag=axF1K1Q6Flmu2pdhgeBJy7yzbVjhUtwmP7FreTNj-9I&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 46,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=320x46&quality=96&crop=15,0,320,46&sign=d11ef321956b2635afed7a1c7b0e77cd&c_uniq_tag=3iA_foo9KxuLLV5y39LHlvfcJIG351rBu3fER64CmV0&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 46,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=350x46&quality=96&sign=8a72abdf87236aec2766b3cc252dc6b3&c_uniq_tag=3M0uqaQu7rL0AaFO2_2lA6BWXFQ0oecRItz09T-bQ0Q&type=album",
        "type": "r",
        "width": 350
       },
       {
        "height": 10,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=75x10&quality=96&sign=d7927f30ecc130acf477e78c72b414fc&c_uniq_tag=0I0mAq04UlF_MP27lO9xZGs5h5rwRpvtDv9OC2y4ZtE&type=album",
        "type": "s",
        "width": 75
       },
       {
        "height": 46,
        "url": "https://sun1-84.userapi.com/impg/R7BRYej2cyweKGRCJdcM5FC83HxSKou728WgGg/zTPZLzqTsUA.jpg?size=350x46&quality=96&sign=8a72abdf87236aec2766b3cc252dc6b3&c_uniq_tag=3M0uqaQu7rL0AaFO2_2lA6BWXFQ0oecRItz09T-bQ0Q&type=album",
        "type": "x",
        "width": 350
       }
      ],
      "text": "",
      "user_id": 341923672
     }
    }
   ],
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807875,
   "from_id": 111485997,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341502,
   "text": "ет с учетом трат на електричество",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807877,
   "from_id": 116398003,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341527,
   "text": "На какой платформе лучше всего майнить?",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 13,
    "items": [
     {
      "id": 6807916,
      "from_id": 145791250,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614341909,
      "text": "[id116398003|Яша], лучше не майнить",
      "likes": {
       "count": 2,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6807877
     },
     {
      "id": 6807918,
      "from_id": 116398003,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614341978,
      "text": "[id145791250|Павел], ясен хрен, но выхода нет",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 145791250,
      "reply_to_comment": 6807916
     },
     {
      "id": 6807935,
      "from_id": 43027971,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342204,
      "text": "[id116398003|Яша], чел смысл суваться в майнинг с 1 карточкой да и еще и старой скорее всего, ибо новую сейчас днем с огнем хрен найдешь))",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6807918
     },
     {
      "id": 6807948,
      "from_id": 116398003,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342320,
      "text": "[id43027971|Александр], я спросил на какой платформе майнить, ты даже не знаешь что есть что нету у меня, я не спрашивал у вас стоит не стоит мне лезть",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 43027971,
      "reply_to_comment": 6807935
     },
     {
      "id": 6807972,
      "from_id": 134104493,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342505,
      "text": "[id116398003|Yasha], ты про прогу?",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6807877
     },
     {
      "id": 6807973,
      "from_id": 43027971,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342508,
      "text": "[id116398003|Яша], да уже по ответам ясно понятно че ты за фрукт, будь ты на серьезных щах настроен в майнинг, не было бы этих тупых вопросов а че а как.",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6807877
     },
     {
      "id": 6807995,
      "from_id": 143706170,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342671,
      "text": "[id116398003|Яша], начни с криптекса, посмотри че и как. Потом можешь на Найс хэше",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6807948
     },
     {
      "id": 6808013,
      "from_id": 116398003,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342849,
      "text": "[id143706170|Сергей], я с криптекса и начал там проще всего, но так как щас волнения битка, там иногда просто карты в холостую крутят , ощущение что надёжности там ничерта",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 143706170,
      "reply_to_comment": 6807995
     },
     {
      "id": 6808014,
      "from_id": 116398003,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614342858,
      "text": "[id43027971|Александр], спасибо за совет я вас услышал",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 43027971,
      "reply_to_comment": 6807973
     },
     {
      "id": 6808045,
      "from_id": 215323914,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807877
      ],
      "date": 1614343168,
      "text": "[id116398003|Яша], на найсе, криптекс помойка",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 116398003,
      "reply_to_comment": 6808013
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807891,
   "from_id": 630760326,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341714,
   "text": "Единственный выход видимо это консоли, и то, пока на них майнить не научились.",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807892,
   "from_id": 430395305,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341718,
   "text": "Какие 3 месяца, она 90к= 1200 бачей стоит, 300 дней минимум выходит",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 6,
    "items": [
     {
      "id": 6807937,
      "from_id": 128569194,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614342240,
      "text": "[id430395305|Gof], какие 90к",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 430395305,
      "reply_to_comment": 6807892,
      "attachments": [
       {
        "type": "photo",
        "photo": {
         "album_id": -8,
         "date": 1614342240,
         "id": 458113388,
         "owner_id": -72378974,
         "has_tags": false,
         "access_key": "a48f20d1f5793d85af",
         "sizes": [
          {
           "height": 66,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=130x66&quality=96&sign=874290a682ec570ccafb9263c45fa8f1&c_uniq_tag=umzOvi-MW1Fct0yPKaF_CZDuIP3dfmWUuO64vsdmGr4&type=album",
           "type": "m",
           "width": 130
          },
          {
           "height": 87,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=130x87&quality=96&crop=188,0,1182,791&sign=e7276044fe78f78759487abc8debd240&c_uniq_tag=-atRwC3Rwy2hy--WUQ2s92wlhuxZdBKJH5GolrH15Xo&type=album",
           "type": "o",
           "width": 130
          },
          {
           "height": 133,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=200x133&quality=96&crop=185,0,1189,791&sign=4cd6b54bac44b57150174dcff8e5b01b&c_uniq_tag=ZamFbFBRzA3RSQzV5e7lHTA-_pu57Pp-kRIMEomeyYk&type=album",
           "type": "p",
           "width": 200
          },
          {
           "height": 213,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=320x213&quality=96&crop=185,0,1188,791&sign=0aa5530b20744fb86bdf85eb4c3737dd&c_uniq_tag=7FmOKXCTCGhwe1qdk09lKB4NLCBElVzFE1E-WQC0TKc&type=album",
           "type": "q",
           "width": 320
          },
          {
           "height": 340,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=510x340&quality=96&crop=186,0,1187,791&sign=a90911293dfe44611190793fab1e22a9&c_uniq_tag=umWZQX4ENsyFcCRcdmXLzL0ScwyfZ_PEtJwuFDGMtKc&type=album",
           "type": "r",
           "width": 510
          },
          {
           "height": 38,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=75x38&quality=96&sign=bd0270b228de6d3c76264c90e9c49986&c_uniq_tag=yyuChNvvQA7964saHB7lyjQsJRVk3mW7BOehKTBVuB4&type=album",
           "type": "s",
           "width": 75
          },
          {
           "height": 791,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=1559x791&quality=96&sign=0ede9f48caf55f04af00c3fe0237c24d&c_uniq_tag=dmd1gTBFBMrc-V6JEmvFPsRo59evlqahOY1TSezSwaw&type=album",
           "type": "w",
           "width": 1559
          },
          {
           "height": 306,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=604x306&quality=96&sign=2bc7d302d6b2509d1d8ddea75008c143&c_uniq_tag=z_LtFhVm6OZXJ8-1kRQVwVkbb8gpWHY1aXSggkjc-vw&type=album",
           "type": "x",
           "width": 604
          },
          {
           "height": 409,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=807x409&quality=96&sign=2b0c3d73ab599bd60f65eec60f867b02&c_uniq_tag=yI7I15sQWFv87WTOaJbN6oSjZwotdKnMhdOhD9Xg3tk&type=album",
           "type": "y",
           "width": 807
          },
          {
           "height": 649,
           "url": "https://sun1-99.userapi.com/impg/6a4iXMbKP9OAygTLVpJ-o3rbi_ZbPNMd8xKHCA/0n6BzLnhxFM.jpg?size=1280x649&quality=96&sign=a61d1def6acfd89d565e4b4d35cddbac&c_uniq_tag=hUQ6OnV64pp5uGu3PnQSlh1aLCZFI0uS8izTeVa4WpY&type=album",
           "type": "z",
           "width": 1280
          }
         ],
         "text": "",
         "user_id": 128569194
        }
       }
      ]
     },
     {
      "id": 6808034,
      "from_id": 12282640,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614343071,
      "text": "[id128569194|Андрей], где такие цены ?",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 128569194,
      "reply_to_comment": 6807937
     },
     {
      "id": 6808039,
      "from_id": 128569194,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614343093,
      "text": "[id12282640|Тигранчик], днс ночью были",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 12282640,
      "reply_to_comment": 6808034
     },
     {
      "id": 6808047,
      "from_id": 12282640,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614343182,
      "text": "[id128569194|Андрей], уже нету их )",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 128569194,
      "reply_to_comment": 6808039
     },
     {
      "id": 6808053,
      "from_id": 128569194,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614343218,
      "text": "[id12282640|Тигранчик], уже да, мб ещё появятся",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 12282640,
      "reply_to_comment": 6808047
     },
     {
      "id": 6808057,
      "from_id": 12282640,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807892
      ],
      "date": 1614343283,
      "text": "[id128569194|Андрей], надеюсь )",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 128569194,
      "reply_to_comment": 6808053
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807913,
   "from_id": 74963945,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614341893,
   "text": "Майнеры горите в аду. Из за вас дефицит ужасный",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807924,
   "from_id": 11324344,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342054,
   "text": "У перекупов берите и за 6 мес не окупится",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807936,
   "from_id": 246972764,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342226,
   "text": "Паблик пора переименовывать в \"Майнеры против геймеров: великое противостояние\"",
   "likes": {
    "count": 7,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614342226,
      "id": 458113387,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "2bbb5f700406d206c2",
      "sizes": [
       {
        "height": 45,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=130x45&quality=96&sign=02ab7e010e7c783ff88e9f7f114f44da&c_uniq_tag=_sKGb0C0Wwb4wq2w7C5NsT_peA3RXqoGwFnU_4Eq1Gg&type=album",
        "type": "m",
        "width": 130
       },
       {
        "height": 87,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=130x87&quality=96&crop=131,0,287,192&sign=0ef75a246d6b4c59b3c14c456ded9dab&c_uniq_tag=mlSzR1wpKzPYR-81hpVIM16p_mVTe-tADzK_jahO2Zk&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 133,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=200x133&quality=96&crop=130,0,289,192&sign=5d8d4b86de44e7cd5333d160c48a1635&c_uniq_tag=LPJ2ZN5QkujrvxmIAwN5IMA63CXRdclmRzDDTbcqPeY&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 192,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=320x192&quality=96&crop=115,0,320,192&sign=39edf1704ad35c564d33b3800b0e0f4e&c_uniq_tag=YglwFBa9dabQ1XNDXKrp2VECVW-19T6dzhoXJNRo4Wg&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 192,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=510x192&quality=96&crop=20,0,510,192&sign=2178efc0965bb70fb0b4c46baeae631a&c_uniq_tag=5bZ_CIZPxb3I_S4Pwk7zNI7BRWN4NlCCCdfCsuLqsq4&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 26,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=75x26&quality=96&sign=7013b91e70d94eeac452812bb112627d&c_uniq_tag=Vyf6QfFVWfRpU5uWCov5YqVk6y5R_-v88iGDP6q3NHs&type=album",
        "type": "s",
        "width": 75
       },
       {
        "height": 192,
        "url": "https://sun1-23.userapi.com/impg/KO9u7Dp5m4d1NQXLNzrVYqn1Reo0d7jJ_YhGGw/ScYXkKTKCng.jpg?size=550x192&quality=96&sign=16c35b0fad10e0688e3b027ac468f000&c_uniq_tag=52B-jISwJGh1p5wTc8RzT232AbrNMHzPMqL6Xz63LAg&type=album",
        "type": "x",
        "width": 550
       }
      ],
      "text": "",
      "user_id": 246972764
     }
    }
   ],
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6807993,
      "from_id": 621847649,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807936
      ],
      "date": 1614342654,
      "text": "[id246972764|Даниил], ахах",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 246972764,
      "reply_to_comment": 6807936
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807947,
   "from_id": 193938532,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342309,
   "text": "4 бакса на 25 мегахешах, ага, держите в курсе, у меня на 57мх 5.5 бакса",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 3,
    "items": [
     {
      "id": 6807966,
      "from_id": 134104493,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807947
      ],
      "date": 1614342443,
      "text": "[id193938532|Igor], ты читать не умеешь?",
      "likes": {
       "count": 1,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 193938532,
      "reply_to_comment": 6807947
     },
     {
      "id": 6808051,
      "from_id": 215323914,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807947
      ],
      "date": 1614343213,
      "text": "[id193938532|Игорь], примерно так и выходит, когда биток за полтинник",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 193938532,
      "reply_to_comment": 6807947
     },
     {
      "id": 6808241,
      "from_id": 543042890,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807947
      ],
      "date": 1614345071,
      "text": "[id193938532|Игорь], не 25, а 41",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 193938532,
      "reply_to_comment": 6807947
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807952,
   "from_id": 217909697,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342347,
   "text": "Кого, они щас по 500$ + стоят",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807979,
   "from_id": 24521486,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342550,
   "text": "",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614342550,
      "id": 458113394,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "758d3892785b563711",
      "sizes": [
       {
        "height": 130,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=65x130&quality=96&sign=98a2e860cd872b7d2ba8fce5648854b9&c_uniq_tag=5UD8HddHBPzOPcL4tQ9YDYtPT-4bxXf5WDGNRTkIRk8&type=album",
        "type": "m",
        "width": 65
       },
       {
        "height": 260,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=130x260&quality=96&sign=c8534019cf3d478c5f090ed5e96f29bb&c_uniq_tag=L3GSBTQAZ1_eGz2E-2CFNtBuocWYbkBjssRf2uQlLWI&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 400,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=200x400&quality=96&sign=e28aec75c967f41a65ed2b92a6eb4f1c&c_uniq_tag=D1CUXmgu_OFLhXV53MV0LHfdJOmBEBMiuTynYSfoFJU&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 640,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=320x640&quality=96&sign=7e1adbb9de760eb73d86022493e4d15b&c_uniq_tag=_Q_di5v6PgpyB_YVotH8H1F6gozCL3BNV6UOdtl3qHQ&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 900,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=510x900&quality=96&crop=0,0,800,1412&sign=0ee085c6fcbc6285d467cd16e778cbcd&c_uniq_tag=mhv1e0h6jYY3PbmDWSbTPkdPvcA0coGtVU7GAw_0hvY&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 75,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=38x75&quality=96&sign=ef54cd75aa172914a447422afdf7c19a&c_uniq_tag=UYY_FIFC5mRQwjxSPdfBXW6to0MyRTwBaWvpE-iuC5A&type=album",
        "type": "s",
        "width": 37
       },
       {
        "height": 1600,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=800x1600&quality=96&sign=6fd2347d855d725154be2569b6c3228b&c_uniq_tag=3PDQ92wdWd3Zn_qhmWjq82E3-n6qymzNm-9SmajKzys&type=album",
        "type": "w",
        "width": 800
       },
       {
        "height": 604,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=302x604&quality=96&sign=8dd561e20af103a4749c66f441d83269&c_uniq_tag=TwaRnUO0i63YzZ08fa9h-Yg2ydxLs47sQ45IYS2ihkw&type=album",
        "type": "x",
        "width": 302
       },
       {
        "height": 807,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=404x807&quality=96&sign=6c223bf98cb5f3a1b6d44bc036524bba&c_uniq_tag=fnsGo550VkmNNp3u8VGxzCiesx-RTqXqhNGbuB6w5hw&type=album",
        "type": "y",
        "width": 403
       },
       {
        "height": 1080,
        "url": "https://sun1-98.userapi.com/impg/bQGP5Gb916f2FYAHlRKQwpX_D9s6YnMS1VsOEw/oZtzbos-azQ.jpg?size=540x1080&quality=96&sign=7b56833446a974b8221eb61950b88aac&c_uniq_tag=Iei6o9j-uNKV3OfQg9Mee_6xf7k2_cKJc4zt5p-mw5U&type=album",
        "type": "z",
        "width": 540
       }
      ],
      "text": "",
      "user_id": 24521486
     }
    },
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614342550,
      "id": 458113395,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "2218e847bafb83a8b2",
      "sizes": [
       {
        "height": 130,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=65x130&quality=96&sign=e253d7f4567404ba6362f8024414aae4&c_uniq_tag=zXHc5EoEjgdE5h5blgHkLhbY_jTepJM8SoM_lEz5CBg&type=album",
        "type": "m",
        "width": 65
       },
       {
        "height": 260,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=130x260&quality=96&sign=a6be3f61bad6c524d261224848ab7db6&c_uniq_tag=DEjPEH_b4ArNM7BpdMXmq6L-4sRXDo1kFW8rvmnSE5Y&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 400,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=200x400&quality=96&sign=457e5ae43fdbaf9840edef4ce7c2a843&c_uniq_tag=o-GTE73QAs5OoKD-VXAYfhYuHBRRQCoCIScjEQG_PL0&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 640,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=320x640&quality=96&sign=532ec6cd7f3820ac81b5bce0f1088e00&c_uniq_tag=9eOUibZcOXG5uXZrGxvhkv0TI7jSCNoEYCCLNY3WSAw&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 900,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=510x900&quality=96&crop=0,0,800,1412&sign=5174c93f58868feb5254076bc2329e89&c_uniq_tag=iUMvIANV9S0WmdHUrBxkaVggI7bGxAVQ7e7VedP9ooI&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 75,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=38x75&quality=96&sign=c20a7920a92347d298823b919cf32bdf&c_uniq_tag=mxPe_5s9paA4_D8vL1V6bBsjgn1d2nX1enwg7E9IgiA&type=album",
        "type": "s",
        "width": 37
       },
       {
        "height": 1600,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=800x1600&quality=96&sign=e947d7e38c450b3112192cd5d096d0b2&c_uniq_tag=PGLB2-5gkytxuDxneFp4ej-Yxqfi0NqcIBGVbY70fTc&type=album",
        "type": "w",
        "width": 800
       },
       {
        "height": 604,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=302x604&quality=96&sign=f8157c90e1b3bb64a1d34995c5fdfd10&c_uniq_tag=xw_Hn84kBcZ2AeIl1OLBSKQ0KVKGOA3e8Wa8FWnj7eM&type=album",
        "type": "x",
        "width": 302
       },
       {
        "height": 807,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=404x807&quality=96&sign=f9882fc69aa3027e936c662170a36457&c_uniq_tag=uuUvl-7aL6yqij4IbstSY1l2dN6JWAwOq_w1UZMdvj8&type=album",
        "type": "y",
        "width": 403
       },
       {
        "height": 1080,
        "url": "https://sun1-83.userapi.com/impg/tDwPJj_xR9SHKnT-0NW52klPYmtO7tiOZG-Tmw/DDmKvSHMT4E.jpg?size=540x1080&quality=96&sign=52ce95ab3d4cd33d5a72fa73d01d4b95&c_uniq_tag=VE2NRlsEgY2wDhRsGyhU4FeW0O4dc1ZLEMk_R2XkYtc&type=album",
        "type": "z",
        "width": 540
       }
      ],
      "text": "",
      "user_id": 24521486
     }
    }
   ],
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6808143,
      "from_id": 3028859,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807979
      ],
      "date": 1614344045,
      "text": "[id24521486|Николай], нахер этих барыг",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 24521486,
      "reply_to_comment": 6807979
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807981,
   "from_id": 24521486,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342560,
   "text": "",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "photo",
     "photo": {
      "album_id": -8,
      "date": 1614342560,
      "id": 458113396,
      "owner_id": -72378974,
      "has_tags": false,
      "access_key": "1620507812eaa5b57c",
      "sizes": [
       {
        "height": 130,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=65x130&quality=96&sign=aa8ba47bc7d944d0e6192e056ab4649d&c_uniq_tag=F42vZWxJv2AjCF0IpZNTfuhbSXR0qBd8Ozj2XC2jTTY&type=album",
        "type": "m",
        "width": 65
       },
       {
        "height": 260,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=130x260&quality=96&sign=f234f6bc9c3e59092531d33b237899b6&c_uniq_tag=lGvQuYnMv507Hv7jNrmO7g-QdX6UWG4VZEa9vc7xuaQ&type=album",
        "type": "o",
        "width": 130
       },
       {
        "height": 400,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=200x400&quality=96&sign=75ece4dd0fa87a903716c7dc03bf0baa&c_uniq_tag=UXcAb6O51YyKlG2VvbWJBt2TWD29ANLI9yxpXrGVoRU&type=album",
        "type": "p",
        "width": 200
       },
       {
        "height": 640,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=320x640&quality=96&sign=986cd62475da06986ee2b7c4aad2b74c&c_uniq_tag=x3xIm7wZG4_oYU6ang_ug5F0dBlaEa3PhIeoSm8H4zU&type=album",
        "type": "q",
        "width": 320
       },
       {
        "height": 900,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=510x900&quality=96&crop=0,0,800,1412&sign=d06c4ea3493cd5e202dd319cb2f64e3e&c_uniq_tag=vagHQff88N1iRRbCh_w6meL1CP_Pz7KFVDCQ4Nqlkgg&type=album",
        "type": "r",
        "width": 510
       },
       {
        "height": 75,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=38x75&quality=96&sign=e1a3dbe19204e5ba239fcfebcea07cc4&c_uniq_tag=ZaJL8kMvTGItMpSvqIJ6x1tqmdl4ZfN5Ec3GEPqrVDc&type=album",
        "type": "s",
        "width": 37
       },
       {
        "height": 1600,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=800x1600&quality=96&sign=c5904ffe9a381cb92adfafb0c03ddea2&c_uniq_tag=qOYjhXidze8ZNS2dwqVf0y8XfWnIQLDDatXrHzSfUZQ&type=album",
        "type": "w",
        "width": 800
       },
       {
        "height": 604,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=302x604&quality=96&sign=f5cbcaf2127d64f8855548e29a91b9d2&c_uniq_tag=kwBu3eeFqv7Bn4AxBu6XGXgSAoqY4Ic5WisAUINnXX0&type=album",
        "type": "x",
        "width": 302
       },
       {
        "height": 807,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=404x807&quality=96&sign=b982a9715cb7a4b55f03a8a5159d1072&c_uniq_tag=UoVC79aprXg7LOCFa_70C2SHT9drWqwDIFqEU5AapZQ&type=album",
        "type": "y",
        "width": 403
       },
       {
        "height": 1080,
        "url": "https://sun1-88.userapi.com/impg/HUh8RN5nZJugVeXGgCNdJBq785h8TIMGBZ7nBw/Yxex7YQZIOs.jpg?size=540x1080&quality=96&sign=d813a8487d095761688d67ac1db0ef56&c_uniq_tag=Iv4mbrtawmeHVbTrB8e2XEjCTkqs6Ipyfw86JUVMYY8&type=album",
        "type": "z",
        "width": 540
       }
      ],
      "text": "",
      "user_id": 24521486
     }
    }
   ],
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6808218,
      "from_id": 496079614,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807981
      ],
      "date": 1614344884,
      "text": "[id24521486|Николай], дорога, я за 45 взял десять штук)",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 24521486,
      "reply_to_comment": 6807981
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807982,
   "from_id": 24521486,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342569,
   "text": "Это новые цены",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6808027,
      "from_id": 77555827,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6807982
      ],
      "date": 1614342989,
      "text": "[id24521486|Николай], в ДНС от 45",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 24521486,
      "reply_to_comment": 6807982
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807984,
   "from_id": 24521486,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342583,
   "text": "За три месяца окупаются🤣🤣🤣🤣",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6807992,
   "from_id": 132118554,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342643,
   "text": "Не было накакой попытки, это было для путда, им выгодно проодавать карты майнерам.",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808022,
   "from_id": 247941658,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342949,
   "text": "Баян😁😁",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808025,
   "from_id": 414272350,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614342959,
   "text": "Закупил 15 штук для фермы. Карты топ. Потом продам лохам на Авито как новые.",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6808066,
      "from_id": 223500601,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6808025
      ],
      "date": 1614343360,
      "text": "[id414272350|Назар], 😄",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 414272350,
      "reply_to_comment": 6808025
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808038,
   "from_id": 20889161,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343093,
   "text": "Чтот с математикой у ребят не очень за 3 месяца 25к выходит.",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808054,
   "from_id": 293008404,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343223,
   "text": "Я скажу за всех в этом сообществе. Б***ть",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808055,
   "from_id": 192025448,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343245,
   "text": "Какая неожиданность",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808067,
   "from_id": 215323914,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343365,
   "text": "",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "video",
     "video": {
      "access_key": "28a7f27e84b8f6cb8b",
      "can_comment": 1,
      "can_like": 1,
      "can_repost": 1,
      "can_subscribe": 1,
      "can_add_to_faves": 1,
      "can_add": 1,
      "comments": 0,
      "date": 1614343361,
      "description": "",
      "duration": 24,
      "image": [
       {
        "height": 96,
        "url": "https://sun1-21.userapi.com/FW14bXO0LlsMpPHyrCLkLxwjaBhAs-S4mSdPFQ/B9zEcn2I0QU.jpg",
        "width": 130,
        "with_padding": 1
       },
       {
        "height": 120,
        "url": "https://sun1-84.userapi.com/LC_awhf07BrOhc7XOgWaK8T8caXR7ZUoergmug/rJA0ceUS6jY.jpg",
        "width": 160,
        "with_padding": 1
       },
       {
        "height": 240,
        "url": "https://sun1-18.userapi.com/bZhI2kr5dAlHyaQ1ZYpb-hHHesncU3CKOx6xvg/dtYgXJ-z_A8.jpg",
        "width": 320,
        "with_padding": 1
       },
       {
        "height": 450,
        "url": "https://sun1-99.userapi.com/QecS-EoGpaX6P0eAZ0qm7Zc2gp3R4rqZOrlJOw/I-TeemWQgoU.jpg",
        "width": 800,
        "with_padding": 1
       },
       {
        "height": 720,
        "url": "https://sun1-14.userapi.com/znpwTYljErqisf-LXVflFYv927_rS0PuBumbnw/vyvcGajMTHI.jpg",
        "width": 1200
       },
       {
        "height": 192,
        "url": "https://sun1-88.userapi.com/YlZohiWiy2BvXqwzRnX-kLQ6_gRBDzdB2LSWqQ/8QEYNX6xvPk.jpg",
        "width": 320
       },
       {
        "height": 432,
        "url": "https://sun1-23.userapi.com/hqxqlz92GNWOEHBFOr_15xJXmha5zQWqH603zQ/3gDLvBN7ILc.jpg",
        "width": 720
       },
       {
        "height": 614,
        "url": "https://sun1-25.userapi.com/iaEF-29bID2do4katHNUnkJIoW2U-cAjUVfi1A/jcPejd68kSw.jpg",
        "width": 1024
       },
       {
        "height": 2458,
        "url": "https://sun1-85.userapi.com/k2klCbhhKiyVx4AFVSxXge8pMOsV10DdHwb95A/6HCvuB6UjCM.jpg",
        "width": 4096
       }
      ],
      "first_frame": [
       {
        "height": 480,
        "url": "https://sun1-22.userapi.com/d6dJSUeknLSSck_4TbiAJ6AO4RCxwf5_q98O_Q/lZZUTlU12P0.jpg",
        "width": 800
       },
       {
        "height": 768,
        "url": "https://sun1-18.userapi.com/_g7gzkZ4bjaRwNygwlGlnmKkSiGtrPEauxSX4Q/Htr7no12PR8.jpg",
        "width": 1280
       },
       {
        "height": 192,
        "url": "https://sun1-85.userapi.com/RP0z1aCtSJKFWAr5JI4EcNifQKCYTGDlPxkMeg/3C41npeGn2c.jpg",
        "width": 320
       },
       {
        "height": 432,
        "url": "https://sun1-93.userapi.com/tAwjJuA75GWonZGEAvO_6LLBWHRh8TZUSUi3sg/fxA1kwravAU.jpg",
        "width": 720
       },
       {
        "height": 614,
        "url": "https://sun1-85.userapi.com/FJ1QHTUMXQvPKnMETNRUq0g3mNzCydFJOlyv1A/YeI6wrJc50E.jpg",
        "width": 1024
       },
       {
        "height": 2458,
        "url": "https://sun1-15.userapi.com/85m6GAhkwJDYzuF5uePAZ8HtYnAj3rrn-g4TAQ/EOPyOG_awtw.jpg",
        "width": 4096
       },
       {
        "height": 192,
        "url": "https://sun1-85.userapi.com/RP0z1aCtSJKFWAr5JI4EcNifQKCYTGDlPxkMeg/3C41npeGn2c.jpg",
        "width": 320
       },
       {
        "height": 96,
        "url": "https://sun1-29.userapi.com/GMnA6yGxxTP_EgRVVMDHQ0VXVCN6c8rldPROuA/zkEZaiDQhBc.jpg",
        "width": 160
       },
       {
        "height": 78,
        "url": "https://sun1-99.userapi.com/xmobjdaFiCMOBtqZaNvYXPvdVw-1NQtpLfBjOA/DfPiJmtwSSo.jpg",
        "width": 130
       }
      ],
      "width": 400,
      "height": 240,
      "id": 456245951,
      "owner_id": -72378974,
      "user_id": 215323914,
      "title": "VID_20210226_071715_323.mp4",
      "is_favorite": false,
      "track_code": "video_2a39df6eA61R9A5ZD2NKDDhDdiL-7Xkg24JzTLJ5NOuiQ0UE_bs5h0v3Y1wAa0sLOitCF8jfTRXit0JIxRhY",
      "type": "video",
      "views": 22
     }
    }
   ],
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808072,
   "from_id": 136666785,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343460,
   "text": "Мда",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808074,
   "from_id": 498542480,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343488,
   "text": "Может просто пулю в затылок майнерам ;)\nХотя да, убивать животных негуманно - нужна лицензия",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 1,
    "items": [
     {
      "id": 6808202,
      "from_id": 249838559,
      "post_id": 6807820,
      "owner_id": -72378974,
      "parents_stack": [
       6808074
      ],
      "date": 1614344748,
      "text": "[id498542480|Анатолий], Майнеры тут почти не при чем.Они лиш исп0лнители чьей то в0ли а именно производителей карт.Это игра и притом продуманная.Все это моё личное оценочное мнение.",
      "likes": {
       "count": 0,
       "user_likes": 0,
       "can_like": 1,
       "can_publish": true
      },
      "reply_to_user": 498542480,
      "reply_to_comment": 6808074
     }
    ],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808081,
   "from_id": 218573996,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343542,
   "text": "За 3 месяца цена упадет в три раза. Так что майнить год полюбе придется",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808082,
   "from_id": 179728176,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343580,
   "text": "лёгкий способ стать олигархом в 15 лет собрать ферму с 3090",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808108,
   "from_id": 179728176,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343766,
   "text": "дефицит из-за падения производства микро чипов для видеокарт майнеры не при чём они заработали лишь пять лямов баксов тогда как у НВИДИА выручка пять миллиардов",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808125,
   "from_id": 179728176,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343882,
   "text": "кстати разбогатеть можно быстрее если где нить заполучить облачное хранилище у них мощные серваки пашут круглосуточно",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808129,
   "from_id": 18164419,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614343928,
   "text": "Полгода она окупаться будет, математики хреновы. Если конечно её можно достать за 30 кусков, то да, три месяца.",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808141,
   "from_id": 3173235,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614344027,
   "text": "5мес и если ты урвал по 45, и за эти 5м курс не изменится 😁",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808180,
   "from_id": 167707738,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614344438,
   "text": "Надо по талонам уже продавать",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808192,
   "from_id": 249838559,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614344589,
   "text": "",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "attachments": [
    {
     "type": "sticker",
     "sticker": {
      "product_id": 4,
      "sticker_id": 140,
      "images": [
       {
        "url": "https://vk.com/sticker/1-140-64",
        "width": 64,
        "height": 64
       },
       {
        "url": "https://vk.com/sticker/1-140-128",
        "width": 128,
        "height": 128
       },
       {
        "url": "https://vk.com/sticker/1-140-256",
        "width": 256,
        "height": 256
       },
       {
        "url": "https://vk.com/sticker/1-140-352",
        "width": 352,
        "height": 352
       },
       {
        "url": "https://vk.com/sticker/1-140-512",
        "width": 512,
        "height": 512
       }
      ],
      "images_with_background": [
       {
        "url": "https://vk.com/sticker/1-140-64b",
        "width": 64,
        "height": 64
       },
       {
        "url": "https://vk.com/sticker/1-140-128b",
        "width": 128,
        "height": 128
       },
       {
        "url": "https://vk.com/sticker/1-140-256b",
        "width": 256,
        "height": 256
       },
       {
        "url": "https://vk.com/sticker/1-140-352b",
        "width": 352,
        "height": 352
       },
       {
        "url": "https://vk.com/sticker/1-140-512b",
        "width": 512,
        "height": 512
       }
      ]
     }
    }
   ],
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808226,
   "from_id": 22755867,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614344920,
   "text": "Тож успел 10 штучек 3080 закупить под фермы",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808233,
   "from_id": 54747082,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614344996,
   "text": "Да там и так было ясно что эта защита не поможет",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808238,
   "from_id": 613841021,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614345041,
   "text": "Ухахах! 😂😂😂 Майнеры опять соснули х*йца у заднеприводной обезьяны! 😏",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808253,
   "from_id": 346529,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614345495,
   "text": "Кто б сомневался",
   "likes": {
    "count": 0,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808257,
   "from_id": 100081186,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614345576,
   "text": "Только вот при расчетах окупаемости вы забыли, что достать данную видяху за 360$ нереально. В реальности окупаемость будет в районе 5-6 месяцев т.к. стоимость электроэнергии вы тоже не учли.",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  },
  {
   "id": 6808266,
   "from_id": 41594992,
   "post_id": 6807820,
   "owner_id": -72378974,
   "parents_stack": [],
   "date": 1614345803,
   "text": "Это просто был рекламный ход",
   "likes": {
    "count": 1,
    "user_likes": 0,
    "can_like": 1,
    "can_publish": true
   },
   "thread": {
    "count": 0,
    "items": [],
    "can_post": true,
    "show_reply_button": true,
    "groups_can_post": true
   }
  }
 ],
 "t": "2021-02-26T16:36:13.755483",
 "iab": 0,
 "iad": 100
}''')

oco.keys()

In [ ]:
D = pd.DataFrame(oco['items'])
D

In [ ]:
D[D.deleted==True]

In [ ]:
D.dropna(subset=['text'], inplace=True)

Добавление колонок для удобства

In [ ]:
D['d'] = pd.to_datetime(D['date'], unit='s')

D['nlike'] = D.likes.apply(lambda o: 0 if (pd.isna(o)) else o['count'])
D['nrepl'] = D.thread.apply(lambda o: 0 if (pd.isna(o)) else o['count'])

Сбор картинок

In [ ]:
v = D.loc[10].attachments

def _attached_image(v):
    if v is np.nan: return
    ty = v[0]['type']
    if ty=='photo':
        if len(v)>1:
            print(f"! {ty} id ={v[0][ty]['id']} + {len(v)} {ty}s")
        upic = [_p for _p in v[0][ty]['sizes'] if _p['type']=='o'][0]['url']
    else:
        return ty
    return Image(url=upic)

_attached_image(v)

In [ ]:
D['image'] = D.attachments.apply(_attached_image)

In [ ]:
pd.options.display.max_colwidth = 100

отметим агрессивные посты вручную по 100 балльной системе.

In [ ]:
D['agr'] =0 

In [ ]:
D.loc[0:10, ['from_id', 'd', 'nlike', 'nrepl', 'text', 'agr', 'image']]

Агрессивность задаётся в баллах от 0 до 100

От 0 до 30 - Комментарий без агрессии, возможно с шуткой
От 30 до 60 - Агрессивна

Определяется:


In [ ]:
D.loc[33,'agr'] = 95
D.loc[1,'agr'] = 80

In [ ]:
X = D[D.text.notna()].copy()

In [ ]:
y = (D.agr * 0.01).round()
y

In [ ]:
def _tokenize(v):

    # Punctuation removal
    table = str.maketrans(dict.fromkeys(string.punctuation))                   
    sentences = (v.translate(table))

    # " '\n " removal
    words = sentences[:-3] 

    # Tokenization
    words = nltk.word_tokenize(words)

    # shrt words removal & lemmatization & stemming
    words_ = []
    for word in words:
        if len(word) > 2:   
            if not word.isnumeric():                                                  
                word1 = stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v'))          
                words_.append(word1)
    return words_

X['w'] = X.text.apply(_tokenize)

In [ ]:
X[['text','w']].head()


In [ ]:
# train_data__, test_data__, train_labels, test_labels = train_test_split(df['comment'], df['toxic'], test_size = 0.2, random_state = 25)

In [ ]:
# CountVectorizer & TermFrequencies
cvect = CountVectorizer(ngram_range=(1, 1), lowercase='true')   
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf=True)

train_data = X['text'].astype(str).values.tolist()
data1_ = cvect.fit_transform(train_data)
data_ = tfidf_transformer.fit_transform(data1_)

### Data visualization

https://www.scikit-yb.org/en/latest/api/text/freqdist.html

In [ ]:
features = cvect.get_feature_names()

visualizer = FreqDistVisualizer(features=features)
visualizer.fit(data1_)
visualizer.poof();

In [ ]:
tsne = TSNEVisualizer(alpha=.45, colors=['green','orange'], random_state=77)
X_ = tsne.fit_transform(data_, y)
tsne.show();

In [ ]:
tsne.transformer_[0].get_params()

In [ ]:
tsne.transformer_[1].get_params()

In [ ]:
D[['tsne_d1', 'tsne_d2']] = tsne.transformer_[1].embedding_

In [ ]:
sns.scatterplot(x='tsne_d1', y='tsne_d2', data=D, hue='labels', alpha=.05, palette={0:'green', 1:'orange'});
plt.plot([70, 70, 90, 90, 70], [5, 35, 35, 5, 5], 'k--');
plt.gca().add_patch(plt.Rectangle((-23, 65), 15, 20, color='m', lw=3, fill=None, ls='--'));


In [ ]:
D_0 = D[(D.tsne_d1>=-35) & (D.tsne_d1<-15) & (D.tsne_d2>-75) & (D.tsne_d2<-50)]
D_1 = D[(D.tsne_d1>=38) & (D.tsne_d1<50) & (D.tsne_d2>10) & (D.tsne_d2<30)]

In [ ]:
print(f'\n'.join(D_0.comments))


In [ ]:
print(f'\n'.join(D_1.comments))


Посмотрим область в правой средней области.

In [ ]:
D_ = D[(D.tsne_d1>=70) & (D.tsne_d1<90) & (D.tsne_d2>5) & (D.tsne_d2<27)]
D_

In [ ]:
toto = []
toto.extend(D_['preprocessed comments'])

toto2 = []
[toto2.extend(_to) for _to in toto]

toto2

In [ ]:
W = pd.value_counts(toto2)
W

In [ ]:
W.iloc[12:32]

In [ ]:
visualizer = FreqDistVisualizer(D_.index).fit(D_['preprocessed comments'])

In [ ]:

visualizer = FreqDistVisualizer(features=features)
visualizer.fit(data1_)

## Supervised learning method - Naive Bayes

In [ ]:
# Model fitting
model = MultinomialNB(alpha = 0.1)
model.fit(train_data, train_labels);

In [ ]:
# Prediction
prediction = model.predict(test_data)

We are using two scores to compare the results

Accuracy score = $\frac{TP + TN}{Total}$

Fowlkes Mallwows Score = $\frac{TP}{\sqrt(TP+FP)(TP+FN}$ 

In [ ]:
accuracy_score(test_labels, prediction) 

In [ ]:
print(fowlkes_mallows_score(prediction, test_labels))

In [ ]:
models = Pipeline([('CountVect', CountVectorizer()), 
                     ('TermFreq', TfidfTransformer()), 
                     ('NB', MultinomialNB())]) 

parameters = { 'CountVect__ngram_range': [(1, 1), (1, 2), (2, 2),(4,5)], 
              'TermFreq__use_idf': (True, False), 
              'TermFreq__norm': ('l1', 'l2'), 
              'NB__alpha': [1, 1e-1, 1e-2, 1e-3] } 

CrossValFolds = 5
grid_search= GridSearchCV(models, parameters, cv = CrossValFolds, n_jobs = -1) 
grid_search.fit(data4, train_labels)

print(grid_search.best_score_) 
print(grid_search.best_params_)

In [ ]:
print(classification_report(test_labels, prediction))

In [ ]:
confusion_matrix(test_labels, prediction, )

In [ ]:
ax = sns.heatmap(confusion_matrix(test_labels, prediction), annot = np.array([['1857', '78'],['350', '598']]), cmap=plt.cm.Blues, fmt = '')

# Unsupervised methods

## K-means

In [ ]:
kmeans = KMeans(n_clusters=2, init='k-means++', random_state=0).fit(data_)

In [ ]:
kmeans_pred = []
for label in kmeans.predict(data_):
    if label == 0:
        kmeans_pred.append(1)
    else:
        kmeans_pred.append(0)

In [ ]:
fowlkes_mallows_score(kmeans_pred, df['toxic']) 

In [ ]:
def accuracy_score_(labels__, labels___):
    score = 0
    for idx, label in enumerate(labels__):
        if label == labels___[idx]:
            score = score + 1

    return score/len(labels___)

In [ ]:
accuracy_score_(kmeans_pred, df['toxic'])

In [ ]:
print(classification_report(df['toxic'], kmeans_pred))

In [ ]:
confusion_matrix(df['toxic'], kmeans_pred)

In [ ]:
ax = sns.heatmap(confusion_matrix(test_labels, prediction), annot = np.array([['6878', '2708'],['3733', '1093']]), cmap=plt.cm.Blues, fmt = '')

### Elbow method

In [ ]:
Y = []
for k in range(1,10):
    kmean_ = KMeans(n_clusters=k).fit(data_)
    Y.append(kmean_.inertia_)

In [ ]:
X = range(1,10)

plt.figure(figsize=(12,6))
plt.plot(X, Y)
plt.plot(2, Y[2-1], 'gD')

plt.text(2.2, Y[2-1]+50, 'Elbow point', bbox=dict(color='green', alpha=0.8))
plt.text(2.38, Y[2-1]+0, 'k = 2', fontsize = 12)

plt.ylabel('Squared distances sum')
plt.xlabel('No of clusters')
plt.title('Elbow Method')

plt.show()

## DBSCAN

In [ ]:
# Dimensionality reduction
SVD = TruncatedSVD(100)
Pca = SVD.fit_transform(data_)

# Makeing the data positive
scaler = MinMaxScaler().fit(Pca)
data_ = scaler.transform(Pca)

In [ ]:
clustering = DBSCAN(eps=0.9, min_samples=2).fit(data_)

In [ ]:
fowlkes_mallows_score(clustering.labels_, data['toxic'])

In [ ]:
accuracy_score_(clustering.labels_, data['toxic'])

### Parameter Tunning

In [ ]:
# Computing a tabel for parameter comparison

Acc = []
Param = []
X_ = []
print('Clusters \t Acc \t eps \t min_samples')
for eps_ in [ 0.6, 0.8, 0.9]:
    for min_samples_ in range(1, 10):
        clustering = DBSCAN(eps=eps_, min_samples=min_samples_).fit(data_)
        n_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
        print( n_clusters,' \t       ', round(accuracy_score_(clustering.labels_, data['toxic']), 3),'\t ', eps_, '  \t ', min_samples_)
        
        X_.append(accuracy_score_(clustering.labels_, data['toxic']))
        if n_clusters == 2 or n_clusters == 3:
            Acc.append(accuracy_score_(clustering.labels_, data['toxic']))
            Param.append([eps_, min_samples_])

In [ ]:
#print('Best score: ', max(Acc),'\nBest Parameters: eps_ = ', Param[np.argmax(Acc, axis = 0)][0], ', min_sample = ', Param[np.argmax(Acc, axis = 0)][1])#

In [ ]:
Y_ = range(0,27)

plt.figure(figsize=(12,6))
plt.plot(Y_, X_)
plt.plot(22, 0.6602830974188176, 'gD')

plt.text(20.2, 0.635, 'Maximum accuracy', bbox=dict(color='green', alpha=0.8))
plt.text(19.4, 0.615, 'eps_ =  0.9 , min_sample =  1', fontsize = 10)

plt.xlabel('Accuracy')
plt.ylabel('Parameters')
plt.title('Best Parameters DBSCAN')

plt.show()